# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [72]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [95]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [96]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [75]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [97]:
# ваш код здесь
data.describe(include = 'object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [98]:
# ваш код здесь
for i in ['Marital Status', 'Gender', 'Home Owner', 'Purchased Bike']:
  k = data.describe(include = 'object')[i].top
  for j in data.index:
    if data[i][j] == k:
      data[i][j] = 1
    else:
      data[i][j] = 0
data

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,1,0,40000,1,Bachelors,Skilled Manual,1,0,0-1 Miles,Europe,42,1
1,24107,1,1,30000,3,Partial College,Clerical,1,1,0-1 Miles,Europe,43,1
2,14177,1,1,80000,5,Partial College,Professional,0,2,2-5 Miles,Europe,60,1
3,24381,0,1,70000,0,Bachelors,Professional,1,1,5-10 Miles,Pacific,41,0
4,25597,0,1,30000,0,Bachelors,Clerical,0,0,0-1 Miles,Europe,36,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,23731,1,1,60000,2,High School,Professional,1,2,2-5 Miles,North America,54,0
996,28672,0,1,70000,4,Graduate Degree,Professional,1,0,2-5 Miles,North America,35,0
997,11809,1,1,60000,2,Bachelors,Skilled Manual,1,0,0-1 Miles,North America,38,0
998,19664,0,1,100000,3,Bachelors,Management,0,3,1-2 Miles,North America,38,1


Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [99]:
# ваш код здесь
data = data.drop(columns = ['Education', 'Occupation', 'Commute Distance', 'Region'])
data

,ID,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,12496,1,0,40000,1,1,0,42,1
1,24107,1,1,30000,3,1,1,43,1
2,14177,1,1,80000,5,0,2,60,1
3,24381,0,1,70000,0,1,1,41,0
4,25597,0,1,30000,0,0,0,36,0
...,...,...,...,...,...,...,...,...,...
995,23731,1,1,60000,2,1,2,54,0
996,28672,0,1,70000,4,1,0,35,0
997,11809,1,1,60000,2,1,0,38,0
998,19664,0,1,100000,3,0,3,38,1


Удалите столбец `ID`, так как он по сути нечисловой.

In [100]:
# ваш код здесь
data = data.drop(columns = ['ID'])
data

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,1,0,40000,1,1,0,42,1
1,1,1,30000,3,1,1,43,1
2,1,1,80000,5,0,2,60,1
3,0,1,70000,0,1,1,41,0
4,0,1,30000,0,0,0,36,0
...,...,...,...,...,...,...,...,...
995,1,1,60000,2,1,2,54,0
996,0,1,70000,4,1,0,35,0
997,1,1,60000,2,1,0,38,0
998,0,1,100000,3,0,3,38,1


Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [101]:
# ваш код здесь
y = data[['Purchased Bike']]
X = data.drop(columns = ['Purchased Bike'])

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [102]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=42) # ваш код здесь

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [52]:
# ваш код здесь
Xtrain.shape

(700, 7)

In [53]:
ytrain.shape

(700, 1)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [103]:
from sklearn.preprocessing import MinMaxScaler

# ваш код здесь

scaler = MinMaxScaler()
scaler.fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [38]:
data.columns[:-1]

Index(['Marital Status', 'Gender', 'Income', 'Children', 'Home Owner', 'Cars',
       'Age'],
      dtype='object')

In [104]:
Xtrain = pd.DataFrame(Xtrain, columns = data.columns[:-1])
Xtrain.head()

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age
0,1.0,1.0,0.2500,0.2,1.0,0.00,0.140625
1,0.0,1.0,0.5000,0.0,0.0,0.75,0.140625
2,0.0,1.0,0.1875,0.0,0.0,0.00,0.171875
3,0.0,1.0,0.6250,0.0,0.0,0.75,0.109375
4,1.0,1.0,0.5000,1.0,1.0,0.75,0.250000


In [105]:
Xtest = pd.DataFrame(Xtest, columns = data.columns[:-1])
Xtest.head()

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age
0,0.0,1.0,0.1875,0.8,1.0,0.5,0.578125
1,1.0,1.0,0.3750,0.4,0.0,0.5,0.375000
2,1.0,1.0,0.1875,0.8,0.0,0.0,0.078125
3,1.0,0.0,0.3125,0.2,1.0,0.0,0.171875
4,1.0,1.0,0.3750,1.0,1.0,0.5,0.281250


Теперь обучите логистическую регрессию на тренировочных данных

In [57]:
print(Xtrain.shape)
print(ytrain.shape)

(700, 7)
(700, 1)


In [106]:
ytrain=ytrain.astype('int')
ytrain

,Purchased Bike
541,1
440,0
482,0
422,0
778,1
...,...
106,0
270,0
860,1
435,1


In [107]:
ytest=ytest.astype('int')
ytest

,Purchased Bike
521,0
737,1
740,1
660,0
411,1
...,...
468,1
935,0
428,1
7,0


In [108]:
from sklearn.linear_model import LogisticRegression

# ваш код здесь

lr = LogisticRegression()
lr.fit(Xtrain, ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [109]:
# ваш код здесь
pr_tr = lr.predict(Xtrain)
pr_tst = lr.predict(Xtest)

Оцените значение accuracy на трейне и на тесте.

In [110]:
from sklearn.metrics import accuracy_score

# ваш код здесь
print(accuracy_score(ytrain, pr_tr))
print(accuracy_score(ytest, pr_tst))

0.6342857142857142
0.58


Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [92]:
data.columns[:-1]

Index(['Marital Status', 'Gender', 'Income', 'Children', 'Home Owner', 'Cars',
       'Age'],
      dtype='object')

In [111]:
from sklearn.preprocessing import PolynomialFeatures

# ваш код здесь
poly = PolynomialFeatures(degree = 2)
poly.fit(Xtrain)
Xtrain = poly.transform(Xtrain)
Xtest = poly.transform(Xtest)
Xtrain = pd.DataFrame(Xtrain)
Xtest = pd.DataFrame(Xtest)

**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

In [112]:
Xtrain.shape

(700, 36)

In [113]:
36-7

29

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [114]:
# ваш код здесь
lr = LogisticRegression()
lr.fit(Xtrain, ytrain)
pr_tr = lr.predict(Xtrain)
pr_tst = lr.predict(Xtest)
print(accuracy_score(ytrain, pr_tr))
print(accuracy_score(ytest, pr_tst))

0.6871428571428572
0.6233333333333333


**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [115]:
# ваш код здесь
from sklearn.metrics import recall_score

print(accuracy_score(ytrain, pr_tr))
print(recall_score(ytrain, pr_tr))


0.6871428571428572
0.738544474393531


Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [116]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [ ]:
lrS = LogisticRegression()
lrS.fit(XtrainS, ytrainS)
pr_val = lr.predict_proba(Xval)
pr_val1 = pr_val[:,1]

In [123]:
pr_val1[0]

0.8098824899127066

In [126]:
from tqdm.notebook import tqdm

In [139]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1
rec_ = []
acc_ = []
thr_ = np.arange(0, 1, 0.01)
for thr in tqdm(np.arange(0, 1, 0.01)):
  # ваш код здесь
  pr_val1_new = []
  for i in pr_val1:
    if i > thr:
      pr_val1_new.append(1)
    else:
      pr_val1_new.append(0)
  rec = recall_score(yval,pr_val1_new)
  acc = accuracy_score(yval, pr_val1_new)
  rec_.append(rec)
  acc_.append(acc)
df = pd.DataFrame(rec_, columns = ['Recall'])
df['Accuracy'] = acc_
df['thr'] = thr_
RecMax = max(df[df['Accuracy'] >= 0.6]['Recall'])
index = df[df['Recall'] == RecMax]['Accuracy'].index[0]
BestAcc = df['Accuracy'][index]
BestThr = df['thr'][index]
print(BestThr, RecMax, BestAcc)

  0%|          | 0/100 [00:00<?, ?it/s]

0.36 0.9166666666666666 0.6047619047619047


In [131]:
df[df['Recall'] == max(df[df['Accuracy'] >= 0.6]['Recall'])]

,Recall,Accuracy,thr
36,0.916667,0.604762,0.36


Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [141]:
# ваш код здесь
lrnew = LogisticRegression()
lrnew.fit(Xtrain, ytrain)
pr_test = lr.predict_proba(Xtest)
pr_test = pr_test[:,1]
pr_test_new = []
for i in pr_test:
    if i > 0.36:
      pr_test_new.append(1)
    else:
      pr_test_new.append(0)
rec = recall_score(ytest,pr_test_new)
acc = accuracy_score(ytest, pr_test_new)
print(acc)
print(rec)

0.5333333333333333
0.8513513513513513


**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.